# Processing Real estate transactions dataset 

In [ ]:
import sys
!{sys.executable} -m pip install geodatasets
!{sys.executable} -m pip install folium>=0.12
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install mapclassify
!{sys.executable} -m pip install h3

In [ ]:
import os
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import geodatasets
import h3
from shapely.geometry import Polygon

## Downloading directly 

In [ ]:
# The data contains lots of information (see below). We keep only the relevant ones for our needs 
relev_cols=['anneemut','datemut','valeurfonc','coddep','sbati','sterr','geompar_x','geompar_y','libtypbien']

In [ ]:
# WARNING : the file is relatively large, so this can take some time
# Online version 
url = 'https://minio.lab.sspcloud.fr/tomvxz/diffusion/transac.parquet'
df_reduced = pd.read_parquet(url, columns=relev_cols)

In [ ]:
# Offline version if available. Run this if you have downloaded the file localy already for future uses
#df_reduced = pd.read_parquet('transac.parquet',columns=relev_cols)

## Dictionary of variables - Dataset DV3F (Cerema)


Explaination from the variables come from here, they were formatted by ChatGPT : https://doc-datafoncier.cerema.fr/doc/dv3f/mutation. Most of them are useless in this case

This dataset comes from the **French Property Transaction Database (DVF – Demandes de Valeurs Foncières)** processed by **Cerema**. It records **real estate transactions carried out for consideration** (sales, exchanges, auctions).

### 1. Identifiers and References
These keys ensure row uniqueness or allow joins with other datasets (Land Registry / MAJIC files).

| Column | Description |
| :--- | :--- |
| **`idmutation`** | Unique transaction identifier (generated by Cerema). |
| `idmutinvar` | Stable identifier used to join with Land Registry (MAJIC) files. |
| `idopendata` | Original identifier from the raw DVF file (DGFiP). |
| `codservch` | Code of the Land Registration Office (SPF). |
| `refdoc` | Publication reference (Volume / Number). |

### 2. Transaction Date and Nature

| Column | Description |
| :--- | :--- |
| **`datemut`** | Exact date of the transaction. |
| `anneemut` | Year of the transaction. |
| `moismut` | Month of the transaction. |
| `idnatmut` | Code describing the nature of the transaction (1 = Sale, etc.). |
| **`libnatmut`** | Label describing the nature (e.g. *Sale*, *Sale of a property under construction*). |
| **`vefa`** | Boolean indicator: Sale of a Property Under Construction (new housing). |
| `nbartcgi` | Number of tax code articles applied. |
| `l_artcgi` | List of applicable tax code articles. |
| `nbdispo` | Number of legal provisions included in the deed. |

### 3. Geographic Location

| Column | Description |
| :--- | :--- |
| `coddep` | Department code (e.g. 75, 59). |
| `nbcomm` | Number of municipalities involved. |
| `l_codinsee` | List of INSEE municipality codes. |
| `nbsection` / `l_section` | Number and list of cadastral sections. |
| `nbpar` / `l_idpar` | Number and list of parcel identifiers involved. |
| `nbparmut` / `l_idparmut` | Number and list of parcels **actually transferred**. |
| `geompar_x` / `geompar_y` | Geographic coordinates (centroids). |

### 4. Price and Land (Plot Characteristics)

| Column | Description |
| :--- | :--- |
| **`valeurfonc`** | **Total transaction value** (net of seller). *Sum for all properties included in the transaction.* |
| `sterr` | Total land area (m²). |
| `l_dcnt` | List of cadastral land areas. |
| `nbsuf` | Number of land surfaces (link with Land Registry files). |

### 5. Building Characteristics (General)

| Column | Description |
| :--- | :--- |
| `codtypbien` | Property typology code (computed by Cerema). |
| **`libtypbien`** | Typology label (e.g. *House*, *Existing apartment*, *Building land*). |
| **`sbati`** | Total built surface area (m²) for all premises. |
| `nblot` | Number of condominium lots. |
| `nbvolmut` | Number of volumes transferred. |
| `nblocmut` | Total number of premises transferred. |
| `l_idlocmut` | List of premises identifiers. |

### 6. Breakdown by Type of Premises
Distribution of premises by category and associated surface areas.

| Column (Count) | Column (Surface) | Description |
| :--- | :--- | :--- |
| `nblocmai` | `sbatmai` | Number and built area of **Houses**. |
| `nblocapt` | `sbatapt` | Number and built area of **Apartments**. |
| `nblocact` | `sbatact` | Number and built area of **Commercial / Activity premises**. |
| `nblocdep` | – | Number of isolated **Outbuildings**. |

### 7. Breakdown by Number of Rooms (Housing Units)
Detail by number of main rooms.  
*Note: `5pp` corresponds to “5 rooms or more”.*

| Type | Counts | Surface Areas (m²) |
| :--- | :--- | :--- |
| **Apartments** | `nbapt1pp` to `nbapt5pp` | `sapt1pp` to `sapt5pp` |
| **Houses** | `nbmai1pp` to `nbmai5pp` | `smai1pp` to `smai5pp` |

---

> **Important note for analysis:**  
> The **`valeurfonc`** variable represents the **total value of the transaction** (`idmutation`).  
> If a single row includes multiple properties (e.g. a building with `nblocapt = 10` apartments), the recorded value corresponds to the **total price of the building**, not the price per apartment.


## Cleaning the dataset

### Removing weird values

In [ ]:
#Checking for na values and non-relevant ones 
na_price = df_reduced['valeurfonc'].loc[df_reduced['valeurfonc'].isna()].count()
print('Nan price: ' + na_price.astype(str))

na_price = df_reduced['valeurfonc'].loc[df_reduced['valeurfonc']==0].count()
print('Null price: ' + na_price.astype(str))

zero_surf = df_reduced['sbati'].loc[df_reduced['sbati']==0].count()
print('Transactions with 0 m2: '+ zero_surf.astype(str))

nbr_trans = df_reduced['valeurfonc'].count()
print('Number of transactions: '+nbr_trans.astype(str))

This is a lot of transactions, but with 0 as the size of the built part, there are either errors or terrains such as fields. So we decide to remove them. Keeping them creates also very weird results in what follows, which is expected.

In [ ]:
# Following previous anaylysis, we clean the data 

# Removing irrelevant transactions based on price
view_price = df_reduced.loc[:,'valeurfonc']
view_geo = df_reduced.loc[:,'geompar_x']
view_surf = df_reduced.loc[:,'sbati'] 
filter =  (view_price.isna()) | (view_geo.isna()) | (view_surf == 0) | (view_price == 0)
df_cleaned = df_reduced[~filter].copy()


In [ ]:
# We check the other columns 
for col in relev_cols:
    print(df_cleaned[col].isna().sum())

In [ ]:
# We check for weird case that could happen else than na values
print(df_cleaned['anneemut'].min(),df_cleaned['anneemut'].max()) #Checking the range of years
print(df_cleaned['datemut'].min(),df_cleaned['datemut'].max()) #Checking concistency with years
print(df_cleaned['valeurfonc'].min(),df_cleaned['valeurfonc'].max()) #Checking range of prices
print(df_cleaned['sbati'].min(),df_cleaned['sbati'].max()) #Checkin range of surfaces

We see that the range of prices is very large and still has values close to 0 (which probably corresponds to things like donation), so we want to remove extreme ones.

### Checking the values of department codes

In [ ]:
df_cleaned['coddep'].unique() # Checking coherent dep codes

In [ ]:
df_reduced["coddep"].nunique()
#for a weird reason there seems to be only 97 departments in this dataset, without further time to inquire about the reasons of this, we will just ignore this problem

### Handling the values of transactions to remove extreme cases

We first look at the repartition of the log price of the transactions. The log allows to get a readable graph. 

In [ ]:
# Creating bins with numpy for faster computations
counts, bin_edges = np.histogram(np.log(df_cleaned['valeurfonc']), bins=100)

# We look at a barplot of the log of valeurfonc to have an idea of the values
plt.bar(bin_edges[:-1], counts, width=np.diff(bin_edges), align='edge')
plt.xlim([0,20])
plt.show()

Based on this, we decide to remove transactions of which the log of price is above 15 or below 7.5.

In [ ]:
# Removing extreme price values
df_cleaned = df_cleaned[(np.log(df_cleaned['valeurfonc'])<=15) & (np.log(df_cleaned['valeurfonc'])>=7.5)]

In [ ]:
# Creating bins with numpy for faster computations
counts, bin_edges = np.histogram(np.log(df_cleaned['valeurfonc']), bins=100)

# We look at a barplot of the log of valeurfonc to have an idea of the values
plt.bar(bin_edges[:-1], counts, width=np.diff(bin_edges), align='edge')
plt.show()

In [ ]:
# Creating bins with numpy for faster computations
counts, bin_edges = np.histogram(np.log(df_cleaned['sbati']), bins=100)

# We look at a barplot of the log of valeurfonc to have an idea of the values
plt.bar(bin_edges[:-1], counts, width=np.diff(bin_edges), align='edge')
plt.show()

### Creation of price per meter squared

In [ ]:
# Since valeurfonc is highly dependant on the size of the building/house sold, we look at price/m2
# We choose to divide first by the built surface (other approches to come)
df_cleaned.loc[:,'p/m2']=df_cleaned['valeurfonc']/df_cleaned['sbati']

In [ ]:
# Creating bins with numpy for faster computations
counts, bin_edges = np.histogram(np.log(df_cleaned['p/m2']), bins=100)

# We look at a barplot of the log of valeurfonc to have an idea of the values
plt.bar(bin_edges[:-1], counts, width=np.diff(bin_edges), align='edge')
plt.show()

In [ ]:
# Filtering the values based on the graph
filter_pm2 = (np.log(df_cleaned['p/m2'])>=3) & (np.log(df_cleaned['p/m2'])<=10)
df_cleaned = df_cleaned[filter_pm2]

The repartition makes now much more sense

In [ ]:
# Creating bins with numpy for faster computations
counts, bin_edges = np.histogram(df_cleaned['p/m2'], bins=100)

# We look at a barplot of the log of valeurfonc to have an idea of the values
plt.bar(bin_edges[:-1], counts, width=np.diff(bin_edges), align='edge')
plt.show()

### Usage of building

In [ ]:
df_cleaned['libtypbien'].unique()

We also have access to the nature of the good being traded, which we could potentially use to be more precise.

## Vizualization

We create the geopandas dataframe to easily draw maps. The caveat is that the given coordinates in the dataframe use different systems depending on the zone. For instance, in France, the system used is Lambert 93 and we have to convert that to the classic GPS system that is widely used. 

In [ ]:
# Geopandas

projections = {
    "Metro":  "EPSG:2154",  # Lambert 93 (France entière hors DOM)
    "971":    "EPSG:5490",  # Guadeloupe (UTM 20N)
    "972":    "EPSG:5490",  # Martinique (UTM 20N)
    "973":    "EPSG:2972",  # Guyane (UTM 22N)
    "974":    "EPSG:2975",  # La Réunion (UTM 40S)
    "976":    "EPSG:4471",   # Mayotte
}

parts = []

#Handling each zones (DOM/TOM have specific zones)
for zone, epsg_code in projections.items():
    if zone == "Metro":
        # Everything that is not a DOM/TOM
        subset = df_cleaned.loc[~((df_cleaned['coddep'].str.startswith('97')) | (df_cleaned['coddep'].isin(['13', '26'])))]
    else:
        # Take specific zone
        subset = df_cleaned.loc[df_cleaned['coddep'] == zone]
    
    if not subset.empty:
        gdf_subset = gpd.GeoDataFrame(
            subset,
            geometry=gpd.points_from_xy(subset['geompar_x'], subset['geompar_y']),
            crs=epsg_code
        )
        
        # Converting to GPS coordinates
        gdf_subset = gdf_subset.to_crs("EPSG:4326")
        
        parts.append(gdf_subset)

gdf_final = pd.concat(parts)



In [ ]:
# We check that our geometries are non-empty
print(gdf_final.geometry.is_empty.sum()) 

In [ ]:
gdf_final.sample(100).explore()

## Per department

In [ ]:
# We standardize the department (adding 0 at the begining if only one number : 1 -> 01)
gdf_final["coddep"] = gdf_final["coddep"].str.zfill(2)

### Number of real_estate transactions per person

In [ ]:
# First, we groupby for each department by numbers of transactions
nb_transac_per_dep = gdf_final.groupby("coddep").size().reset_index(name='Number_transactions')
nb_transac_per_dep.to_csv("nb_transac_per_dep.csv")

In [ ]:
# Getting population data from insee
def deps_pop_init():
    deps_pop = pd.read_excel("https://www.insee.fr/fr/statistiques/fichier/2012713/TCRD_004.xlsx")
    deps_pop.columns = deps_pop.iloc[2]
    deps_pop = deps_pop.iloc[3:]
    deps_pop = deps_pop.reset_index()
    deps_pop.columns = ["to_del", "code", "deps_name", "2025", "share_pop","2022", "2016","2011","1999"]
    deps_pop.set_index("code")
    deps_pop = deps_pop.loc[0:101,["code","2011","2025"]]
    return deps_pop

deps_pop = deps_pop_init()

In [ ]:
# Function to merge the data and get population data
def add_department_pop(df):
    df = df.reset_index()
    df = df.merge(
        deps_pop[["code", "2011", "2025"]],
        left_on="coddep",
        right_on="code",
        how="left"
    )
    df = df.drop(columns=["code"]).set_index("coddep")
    return df

In [ ]:
# Adding name to department
deps = gpd.read_file("https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements.geojson")

def add_department_name(df):
    df = df.merge(
        deps[["code", "nom"]],
        left_on="coddep",
        right_on="code",
        how="left"
    )

    df["coddep"] = (
        df["code"].astype(str) + " - " + df["nom"]
    )

    df = (
        df
        .drop(columns=["code", "nom"])
        .set_index("coddep")
    )

    return df

In [ ]:
# Creating transaction frequency
nb_transac_per_dep = nb_transac_per_dep[["coddep","Number_transactions"]]
nb_transac_per_dep.rename(columns={"coddep": "Code du département du lieu des travaux - Code de la zone"})
nb_transac_per_pop_dep = add_department_pop(nb_transac_per_dep)
nb_transac_per_pop_dep["transac_freq"] =  nb_transac_per_pop_dep["Number_transactions"]/nb_transac_per_pop_dep["2025"]


In [ ]:
add_department_name(nb_transac_per_pop_dep["transac_freq"].sort_values(ascending=False).to_frame().head(10))


Number of real estate transactions per population could be an indicator of how intensive is the trading in a place, which could be an indicator of speculation, defined as buying a property to make monetary profit from the variation of price. 
However, population does not take into account secondary residency owned by non resident of a department. 


### Growth of p/m2

In [ ]:
median_pm2_per_dep = gdf_final.groupby("coddep")["p/m2"].median().to_frame()
add_department_name(median_pm2_per_dep["p/m2"].sort_values(ascending=False).to_frame().head(10))

In [ ]:
pm2_growth_per_dep = gdf_final.groupby(["coddep","anneemut"])["p/m2"].median().unstack(1)
pm2_growth_per_dep

In [ ]:
pm2_growth_per_dep = pm2_growth_per_dep.pct_change(axis=1)
pm2_growth_per_dep = pm2_growth_per_dep.quantile(0.9,axis=1).to_frame()
pm2_growth_per_dep

In [ ]:
add_department_name(pm2_growth_per_dep[0].sort_values(ascending=False).to_frame()).head(10)

In [ ]:
pm2_growth_pop_per_dep = add_department_pop(pm2_growth_per_dep)
pm2_growth_pop_per_dep["pop_growth"] = (pm2_growth_pop_per_dep["2025"]-pm2_growth_pop_per_dep["2011"])/pm2_growth_pop_per_dep["2025"]

In [ ]:
plt.scatter(
    pm2_growth_pop_per_dep["pop_growth"],
    pm2_growth_pop_per_dep[0]
)

Departments where there was high increases of prices during the period are not departments that had high level of population growth

What if we look at the "average" (we will here take median as the dataset contains extremely low and extremely high values) growth rate of prices during the period ? 


In [ ]:
pm2_growth_per_dep = gdf_final.groupby(["coddep","anneemut"])["p/m2"].median().unstack(1)
pm2_growth_per_dep = pm2_growth_per_dep.pct_change(axis=1)
pm2_growth_per_dep = pm2_growth_per_dep.quantile(0.5,axis=1).to_frame()
add_department_name(pm2_growth_per_dep[0.5].sort_values(ascending=False).to_frame()).head(10)
pm2_growth_pop_per_dep = add_department_pop(pm2_growth_per_dep)
pm2_growth_pop_per_dep["pop_growth"] = (pm2_growth_pop_per_dep["2025"]-pm2_growth_pop_per_dep["2011"])/pm2_growth_pop_per_dep["2025"]
plt.scatter(
    pm2_growth_pop_per_dep["pop_growth"],
    pm2_growth_pop_per_dep[0.5]
)

In [ ]:
add_department_name(pm2_growth_pop_per_dep.loc[pm2_growth_pop_per_dep[0.5]>0.06])


These are the departments where median price growth over the period was the highest, despite normal or even negative population growth. These departments do not seem to have any distinctive feature

In [ ]:
pm2_growth_per_dep = gdf_final.groupby(["coddep","anneemut"])["p/m2"].median().unstack(1)
pm2_growth_per_dep.iloc[18:45,]


We remark that these are departments where there are not a lot of datas : the outliers seems to be statistical artefact 

In [ ]:
pm2_growth_per_dep = gdf_final.groupby(["coddep","anneemut"])["p/m2"].median().unstack(1)
pm2_growth_per_dep = pm2_growth_per_dep.pct_change(axis=1)
pm2_growth_per_dep = pm2_growth_per_dep.mean(axis=1).to_frame()
add_department_name(pm2_growth_per_dep[0].sort_values(ascending=False).to_frame()).head(10)
pm2_growth_pop_per_dep = add_department_pop(pm2_growth_per_dep)
pm2_growth_pop_per_dep["pop_growth"] = (pm2_growth_pop_per_dep["2025"]-pm2_growth_pop_per_dep["2011"])/pm2_growth_pop_per_dep["2025"]

plt.scatter(
    pm2_growth_pop_per_dep["pop_growth"],
    pm2_growth_pop_per_dep[0]
)

## Per hexagon

In [ ]:
# 1. Configuration

resolution = 7

# ---------------------------------------------------------
# OPTIMISATION 1 : Assigning a hexagon to every transaction 
# ---------------------------------------------------------

# On extrait les colonnes en tableaux NumPy purs (.values)
# C'est beaucoup plus rapide que d'accéder à geometry.x ligne par ligne
lats = gdf_final.geometry.y.values
lons = gdf_final.geometry.x.values

# On utilise une compréhension de liste avec zip
# C'est la méthode standard la plus rapide en Python pur
gdf_final['h3_index'] = [
    h3.latlng_to_cell(lat, lon, resolution) 
    for lat, lon in zip(lats, lons)
]


In [ ]:
# ---------------------------------------------------------
# OPTIMISATION 2 : Reconstructing the geometry of the h3-hexagon
# ---------------------------------------------------------

def hex_to_poly_optimized(h3_id):
    # cell_to_boundary renvoie ((lat, lon), (lat, lon)...)
    points = h3.cell_to_boundary(h3_id)
    # Inversion (Lat, Lon) -> (Lon, Lat) + Création Polygon
    return Polygon([(lng, lat) for lat, lng in points])

def display_hexagon_map(df_counts,variable):
    # Ici aussi, on évite .apply() pour utiliser une boucle de liste directe
    #we create a geodataframe that assign a geometry to every hexagon
    gdf_h3 = gpd.GeoDataFrame(
        df_counts,
        geometry=[hex_to_poly_optimized(h_id) for h_id in df_counts['h3_index'].values],
        crs="EPSG:4326"
    )

    #displaying the hexagon
    m = gdf_h3.explore(
        column=variable,
        cmap="inferno",
        style_kwds={"fillOpacity": 0.6, "weight": 0},
        tiles="CartoDB positron"
    )
    return m

### Number of transactions per hexagon

In [ ]:
variable = "Number_transactions"
df_counts = gdf_final.groupby('h3_index').size().reset_index(name=variable)
display_hexagon_map(df_counts, variable)


### Median price of a transation per hexagon

In [ ]:
variable = "Median price of transactions"
df_counts = gdf_final.groupby('h3_index')['valeurfonc'].median().reset_index(name=variable)
display_hexagon_map(df_counts, variable)

### Median p/m2 per hexagon

In [ ]:
gdf_final = gdf_final[gdf_final['p/m2'] < np.inf]

In [ ]:
variable = "Median p/m2"
df_counts = gdf_final.groupby('h3_index')['p/m2'].median().reset_index(name=variable)
display_hexagon_map(df_counts, variable)

### Time-series analysis 

#### Nb of constructions

In [ ]:
gdf_final.groupby('h3_index').size().reset_index(name='Number_transactions')

#### Median p/m2

In [ ]:
gdf_final

In [ ]:
gdf_final = gdf_final.loc[gdf_final['p/m2']<50000,]


In [ ]:
gdf_final = gdf_final[gdf_final['p/m2'] < np.inf]
gdf_final.head()

In [ ]:
pm2_growth_per_hex = gdf_final.groupby(['h3_index','anneemut'])['p/m2'].median().unstack(1)
pm2_growth_per_hex =pm2_growth_per_hex.pct_change(axis=1)
# log growth : pm2_growth_per_hex = np.log1p(pm2_growth_per_hex).diff(axis=1)

    
pm2_growth_per_hex

In [ ]:
pm2_growth_per_hex = pm2_growth_per_hex.median(axis=1)


In [ ]:
pm2_growth_per_hex.to_csv("pm2_growth_per_hex.csv")